In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Head_and_Neck_Cancer"
cohort = "GSE201777"

# Input paths
in_trait_dir = "../../input/GEO/Head_and_Neck_Cancer"
in_cohort_dir = "../../input/GEO/Head_and_Neck_Cancer/GSE201777"

# Output paths
out_data_file = "../../output/preprocess/Head_and_Neck_Cancer/GSE201777.csv"
out_gene_data_file = "../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE201777.csv"
out_clinical_data_file = "../../output/preprocess/Head_and_Neck_Cancer/clinical_data/GSE201777.csv"
json_path = "../../output/preprocess/Head_and_Neck_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Differentially expressed genes related to lymph node metastasis in advanced laryngeal squamous cell cancers"
!Series_summary	"Understanding the molecular mechanisms and gene expression in laryngeal squamous cell carcinoma (LSCC) may explain its aggressive biological behavior and regional metastasis pathways. Better understanding of the molecular mechanisms underlying LSCC metastasis and the search for possible molecular targets seems promising. Interpreting the links between the differentially expressed genes in advanced stages can lead to a search for predictive markers that can also help determine the possible treatment routes. We designed this study to detect possible genetic alterations in a homogeneous group of patients with locoregionally advanced laryngeal cancer who underwent total laryngectomy and neck dissection. Patients with and without lymph node metastasis were selected to examine the differential gene expression in the normal mucosa

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Determine gene expression data availability 
# From background information, we can see this dataset is about gene expression related to lymph node metastasis in LSCC
is_gene_available = True

# 2. Identify keys for trait, age, and gender and create conversion functions
# Looking at the sample characteristics, we can see:
# - Index 2 contains lymph node metastasis status (can be used as trait)
# - No age information available
# - No gender information available

# 2.1 Data availability
trait_row = 2  # lymph node metastasis information is available at index 2
age_row = None  # No age information available
gender_row = None  # No gender information available

# 2.2 Data type conversion functions
def convert_trait(value):
    """Convert trait (lymph node metastasis) value to binary (0 for negative, 1 for positive)"""
    if not value or not isinstance(value, str):
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value.lower() == "negative":
        return 0
    elif value.lower() in ["positive", "positiive"]:  # Handling typo in the data
        return 1
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric"""
    # Not used as age data is not available, but required for function signature
    return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    # Not used as gender data is not available, but required for function signature
    return None

# 3. Perform initial filtering and save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. If trait data is available, extract clinical features
if trait_row is not None:
    # Assuming clinical_data is already available from a previous step
    # We'll continue with the processing using the existing clinical_data DataFrame
    
    # Use library function to extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait="Lymph_Node_Metastasis",
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    print(preview_df(clinical_features))
    
    # Ensure output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical features to CSV
    clinical_features.to_csv(out_clinical_data_file, index=False)


{'GSM6071161': [0.0], 'GSM6071162': [0.0], 'GSM6071163': [0.0], 'GSM6071164': [0.0], 'GSM6071165': [0.0], 'GSM6071166': [0.0], 'GSM6071167': [0.0], 'GSM6071168': [0.0], 'GSM6071169': [0.0], 'GSM6071170': [1.0], 'GSM6071171': [1.0], 'GSM6071172': [1.0], 'GSM6071173': [1.0], 'GSM6071174': [1.0], 'GSM6071175': [1.0], 'GSM6071176': [0.0], 'GSM6071177': [0.0], 'GSM6071178': [0.0], 'GSM6071179': [0.0], 'GSM6071180': [0.0], 'GSM6071181': [0.0], 'GSM6071182': [0.0], 'GSM6071183': [0.0], 'GSM6071184': [0.0], 'GSM6071185': [0.0], 'GSM6071186': [0.0], 'GSM6071187': [0.0], 'GSM6071188': [1.0], 'GSM6071189': [1.0], 'GSM6071190': [1.0], 'GSM6071191': [1.0], 'GSM6071192': [1.0], 'GSM6071193': [1.0], 'GSM6071194': [1.0], 'GSM6071195': [1.0], 'GSM6071196': [1.0], 'GSM6071197': [1.0], 'GSM6071198': [1.0], 'GSM6071199': [0.0], 'GSM6071200': [0.0], 'GSM6071201': [0.0], 'GSM6071202': [1.0], 'GSM6071203': [1.0], 'GSM6071204': [1.0], 'GSM6071205': [1.0], 'GSM6071206': [1.0], 'GSM6071207': [1.0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Head_and_Neck_Cancer/GSE201777/GSE201777_series_matrix.txt.gz


Gene data shape: (49395, 47)


First 20 gene/probe identifiers:
Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, they appear to be Affymetrix probe IDs (with the "_at", "_s_at", "_x_at" pattern)
# rather than standard human gene symbols like "TP53" or "BRCA1"
# These probe IDs will need to be mapped to human gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'GeneChip Array', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Transcript ID(Array Design)', 'Target Description', 'GB_ACC', 'GI', 'Representative Public ID', 'Archival UniGene Cluster', 'UniGene ID', 'Genome Version', 'Alignments', 'Gene Title', 'Gene Symbol', 'Chromosomal Location', 'Unigene Cluster Type', 'Ensembl', 'Entrez Gene', 'SwissProt', 'EC', 'OMIM', 'RefSeq Protein ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function', 'Pathway', 'InterPro', 'Annotation Description', 'Annotation Transcript Cluster', 'Transcript Assignments', 'Annotation Notes', 'SPOT_ID']
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe gene annotation data to determine which columns to use for mapping
print("\nDetermining columns for gene mapping:")
print(f"Gene expression data has identifiers like: {gene_data.index[0:3]}")
print(f"Gene annotation data has 'ID' column with values like: {gene_annotation['ID'][0:3]}")
print(f"Gene annotation data has 'Gene Symbol' column with values like: {gene_annotation['Gene Symbol'][0:3]}")

# We need to map from probe IDs (in column 'ID') to gene symbols (in column 'Gene Symbol')
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get the gene mapping dataframe by extracting relevant columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"\nGene mapping dataframe shape: {gene_mapping.shape}")
print("Gene mapping sample (first 5 rows):")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"\nMapped gene expression data shape: {gene_data.shape}")
print("Gene expression data sample (first 5 genes, first 3 samples):")
print(gene_data.iloc[:5, :3])

# Save the gene data to CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to: {out_gene_data_file}")



Determining columns for gene mapping:
Gene expression data has identifiers like: Index(['11715100_at', '11715101_s_at', '11715102_x_at'], dtype='object', name='ID')
Gene annotation data has 'ID' column with values like: 0      11715100_at
1    11715101_s_at
2    11715102_x_at
Name: ID, dtype: object
Gene annotation data has 'Gene Symbol' column with values like: 0    HIST1H3G
1    HIST1H3G
2    HIST1H3G
Name: Gene Symbol, dtype: object

Gene mapping dataframe shape: (49372, 2)
Gene mapping sample (first 5 rows):
              ID       Gene
0    11715100_at   HIST1H3G
1  11715101_s_at   HIST1H3G
2  11715102_x_at   HIST1H3G
3  11715103_x_at  TNFAIP8L1
4  11715104_s_at      OTOP2



Mapped gene expression data shape: (19963, 47)
Gene expression data sample (first 5 genes, first 3 samples):
         GSM6071161  GSM6071162  GSM6071163
Gene                                       
A1BG       2.000830    1.656920    2.373500
A1CF       3.209655    3.271378    3.212505
A2M        1.958110    3.177790    2.807670
A2ML1      4.446110    3.589280    4.977190
A3GALT2    1.632900    1.721840    2.705260



Gene expression data saved to: ../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE201777.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        # Check if trait is biased
        trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
        if trait_type == "binary":
            is_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            is_biased = judge_continuous_variable_biased(linked_data, trait)
            
        # Remove biased demographic features
        if "Age" in linked_data.columns:
            age_biased = judge_continuous_variable_biased(linked_data, 'Age')
            if age_biased:
                linked_data = linked_data.drop(columns='Age')
                
        if "Gender" in linked_data.columns:
            gender_biased = judge_binary_variable_biased(linked_data, 'Gender')
            if gender_biased:
                linked_data = linked_data.drop(columns='Gender')
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to Randall's plaque tissue, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data from Randall's plaque tissue associated with kidney stones."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for kidney stones analysis."
    )
    
    print("Dataset is not usable for kidney stones analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (19963, 47)
Gene data shape after normalization: (19758, 47)


Normalized gene expression data saved to ../../output/preprocess/Head_and_Neck_Cancer/gene_data/GSE201777.csv
Original clinical data preview:
         !Sample_geo_accession  \
0  !Sample_characteristics_ch1   
1  !Sample_characteristics_ch1   
2  !Sample_characteristics_ch1   

                                          GSM6071161  \
0  patient diagnosis: laryngeal squamous cell car...   
1                                 tissue: Lymph node   
2                    lymph node metastasis: negative   

                                          GSM6071162  \
0  patient diagnosis: laryngeal squamous cell car...   
1                                      tissue: Tumor   
2                    lymph node metastasis: negative   

                                          GSM6071163  \
0  patient diagnosis: laryngeal squamous cell car...   
1                                     tissue: Mucosa   
2                    lymph node metastasis: negative   

                                          GSM6

Data shape after handling missing values: (47, 19759)
For the feature 'Head_and_Neck_Cancer', the least common label is '1.0' with 23 occurrences. This represents 48.94% of the dataset.


Linked data saved to ../../output/preprocess/Head_and_Neck_Cancer/GSE201777.csv
